We have used the following tech stack and concepts
1) Langchain
2) RAG-addition of our own data
3) OPEN AI Model
4) Mistral Model

We explored 2 LLM's on our sample training data set. We wanted to observe the effectiveness of each model on our data

Langchain components

In [ ]:
!pip install -q langchain openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


OPEN AI - CHAT GPT version

In [1]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import pandas as pd
import os

os.environ["OPENAI_API_KEY"] = ""



MessageError: ignored

In [ ]:

file_path = './data/demo.csv'

# Load the dataset
loader = CSVLoader(file_path=file_path)


In [ ]:
loader

In [ ]:

index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([loader])

NameError: ignored

In [ ]:
# Create a question-answering chain using the index
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

3 Query tests on OPEN AI model

In [ ]:
query = "if my danceability is 9, what artist should I collaborate with"
response = chain({"question": query})
print(response['result'])

 It looks like Calvin Harris has a collaboration score of 89 with a danceability of 9, so you may want to consider collaborating with him.


In [ ]:
query = "if my danceability is 8, what artist should I collaborate with"
response = chain({"question": query})
print(response['result'])

 I don't know.


In [ ]:
query = "if my danceability is 8, what artist would be close to this, what artist should I collaborate with, recommend the artist and give all reasons"
response = chain({"question": query})
print(response['result'])

 You might want to consider collaborating with Calvin Harris. His songs have a danceability score of 9 and a collab_score of 89, which is close to your desired danceability of 8 and collab_score of 60.


**Mistral version**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Above is FIX for Unicode Error in Colab

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
# Create a question-answering chain using the index
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

3 query tests on Mistral model

In [ ]:
query = "if my danceability is 9, what artist should I collaborate with"
response = chain({"question": query})
print(response['result'])

 Based on your danceability of 9, you may want to consider collaborating with Calvin Harris or B61 as both artists have high danceability scores as well (89 and 60 respectively). Madonna, however, has the lowest danceability score among all the artists listed here. So, if you are looking for someone to collaborate with that matches your danceability level, Calvin Harris and B61 could be good options.


In [ ]:
query = "if my danceability is 8, what artist should I collaborate with"
response = chain({"question": query})
print(response['result'])

 Based on the given context, it appears that the artist with the highest collaboration score (Calvin Harris) has a relatively low loudness score, indicating a more subdued and sophisticated sound. On the other hand, the artist with a high collaboration score and high loudness score (Madonna) is likely to have a very loud and intense sound. Therefore, if your danceability is 8, you may want to collaborate with an artist like Calvin Harris, who can provide you with both high danceability and a more subdued sound that is still sophisticated and polished.


In [ ]:
query = "if my danceability is 8, what artist would be close to this, what artist should I collaborate with, recommend the artist and give all reasons"
response = chain({"question": query})
print(response['result'])

 Based on the given information, Calvin Harris is a great option for collaborations as he has a high collab_score of 89 and his danceability score is also relatively high with 9. Madonna, on the other hand, has a very low danceability score of 6, but her collab_score is 20, which is a good indication of her collaborative skills. B61, with a danceability score of 4 and a collab_score of 60, could be a decent collaboration partner but may not have the same level of popularity as Calvin Harris or Madonna.


We didn't have time to explore other models and how best to evaluate LLM models. This is something we would like to explore further, there was a open source farmework found(https://docs.ragas.io/en/latest/concepts/metrics_driven.html) to mention a logical and standardized way to explore and evaluate LLM results. Most of the exploration above is based on Human and manual interpretation

There is also a recent application called Langsmith which is also another open source LLM evaluation tool